![1](images/TENSOR_TUNDRA_200_1.png)
    
![2](images/TENSOR_TUNDRA_200_2.png)

![3](images/TENSOR_TUNDRA_200_3.png)
    
<!-- ![4](images/TENSOR_TUNDRA_200_4.png) -->

In [1]:
import json
import pennylane as qml
import pennylane.numpy as np


dev = qml.device("default.qubit", wires=["player1", "player2", "goalie"])

# Put any extra functions you want here
def quantum_circuit(params, t_wire):
    qml.Rot(*params, wires=t_wire)
def state_prep(player_coeffs, goalie_coeffs):
    """
    Contains quantum operations that prepare |psi> and |phi>. We recommend using
    qml.StatePrep!

    Args:
        - player_coeffs (list(float)): 
            The coefficients, alpha, beta, and kappa (in that order) that describe 
            the quantum state of players 1 and 2:

            |psi> = alpha|01> + beta|10> + kappa(|00> + |11>)

        - goalie_coeffs (list(float)):
            The coefficients, gamma and delta (in that order) that describe 
            the quantum state of the goalie:

            |phi> = gamma|0> + delta|1>
    """

    alpha, beta, kappa = player_coeffs
    gamma, delta = goalie_coeffs

    # Put your code here #
    qml.StatePrep(np.array([kappa, alpha, beta, kappa]), wires=["player1", "player2"])
    qml.StatePrep(np.array([gamma,delta]), wires=["goalie"])

@qml.qnode(dev)
def save_percentage(player_coeffs, goalie_coeffs, x, y, z):
    """
    Calculates the save percentage of the goalie.

    NOTE: This QNode may only contain 7 operations or less (counting any 
    operations used in the state_prep function) and must use three conditional
    measurements (i.e., 3 instances of qml.cond).

    Args:
        - player_coeffs (list(float)): 
            The coefficients, alpha, beta, and kappa (in that order) that describe 
            the quantum state of players 1 and 2:

            |psi> = alpha|01> + beta|10> + kappa(|00> + |11>)

        - goalie_coeffs (list(float)):
            The coefficients, gamma and delta (in that order) that describe 
            the quantum state of the goalie:

            |phi> = gamma|0> + delta|1>
        
        - x, y, z (float): 
            The amounts that affect the goalie's save percentage based on 
            measuring the players.

    Returns:
        - (numpy.tensor): The save percentage of the goalie.
    """
    state_prep(player_coeffs, goalie_coeffs)

    # Put your code here #

    gamma, delta = goalie_coeffs

    def true_fn1():
        
        U1 = [[1.0/gamma, 0.0], [0.0, 1.0/delta]] 
        U2 = [[np.sqrt(x)*gamma, 0.0], [0.0, np.sqrt(1-(np.sqrt(x)*gamma)**2)]]
        U = np.dot(U2,U1)
        qml.QubitUnitary(U, wires=["goalie"])
        # qml.QubitUnitary(U2, wires=["goalie"])
        
        
    def true_fn2():
        U1 = [[1.0/gamma, 0.0], [0.0, 1.0/delta]] 
        U2 = [[np.sqrt(y)*gamma, 0.0], [0.0, np.sqrt(1-(np.sqrt(y)*gamma)**2)]]
        U = np.dot(U2,U1)
        qml.QubitUnitary(U, wires=["goalie"])
        # qml.QubitUnitary(U2, wires=["goalie"])
        
    def true_fn3():
        U1 = [[1.0/gamma, 0.0], [0.0, 1.0/delta]] 
        U2 = [[np.sqrt(1-(np.sqrt(z)*delta)**2), 0.0], [0.0, np.sqrt(z)*delta]]
        U = np.dot(U2,U1)
        qml.QubitUnitary(U, wires=["goalie"])
        # qml.QubitUnitary(U2, wires=["goalie"])
        
    m0 = qml.measure(wires =["player1"])
    m1 = qml.measure(wires =["player2"])
        
    qml.cond(m0 & ~m1 == 1, true_fn1 )()
    qml.cond(~m0 & m1 == 1, true_fn2)()
    qml.cond(~((m0 | m1) & (~ (m0 & m1))) == 1, true_fn3 )()
# qml.cond(m_0==1, qml.cond(m_1==1 , true_fn3, true_fn1), qml.cond(m_1==1 , true_fn2, true_fn3))
    
    # qml.StatePrep(np.array([new_gamma, new_delta]), wires=["goalie"])
    return qml.probs(wires="goalie")

# These functions are responsible for testing the solution.
def run(test_case_input: str) -> str:
    ins = json.loads(test_case_input)
    player_coeffs, goalie_coeffs, x, y, z = ins
    output = save_percentage(player_coeffs, goalie_coeffs, x, y, z).tolist()
    return str(output)

def check(solution_output: str, expected_output: str) -> None:
    solution_output = json.loads(solution_output)
    sp = solution_output
    _sp = json.loads(expected_output)

    ops = save_percentage.tape._ops
    num_ops = len(ops)
    num_cond = [op.name for op in ops].count('Conditional')
    names = [op.name for op in ops]
    state_prep_check = ('StatePrep' or 'MottonenStatePreparation' or 'AmplitudeEmbedding') in names

    assert np.allclose(sp, _sp, rtol=1e-4), "Your calculated save percentage is incorrect."
    assert num_ops < 8, "You used more than 7 operations in your save_percentage function."
    assert num_ops > 2, "You definitely need more than 2 operations..."
    assert state_prep_check, "You can use StatePrep, MottonenStatePreparation, or AmplitudeEmbedding to prepare states."
    assert num_cond == 3, "You haven't used exactly 3 qml.cond operators."
